In [1]:
import os
from Perplexity import Perplexity
from supabase import create_client, Client
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.vectorstores import SupabaseVectorStore
from langchain.document_loaders import TextLoader
from langchain.document_loaders.csv_loader import CSVLoader
from dotenv import load_dotenv

load_dotenv()

HOSTNAME = os.environ.get("HOSTNAME")
PORT = os.environ.get("PORT")
PASSWORD = os.environ.get("PASSWORD")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
SUPABASE_URL = os.environ.get("SUPABASE_URL")
SUPABASE_SERVICE_KEY = os.environ.get("SUPABASE_SERVICE_KEY")

2023-07-29 17:21:59,902:INFO - NumExpr defaulting to 8 threads.
/Users/yecao/anaconda3/envs/opennice_infra/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.14) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [11]:
import redis
import json
import pandas as pd

def get_all_failed_queries():
    def connect_reddis():
        return redis.Redis(
            host=HOSTNAME,
            port=PORT, 
            password=PASSWORD,
            ssl=True,  # Enable SSL/TLS encryption for the connection
            decode_responses=True  # Convert responses to strings (optional but useful for human-readable output)
        )

    # Connect KV db
    r = connect_reddis()

    # Get all chat keys by pattern
    all_keys = r.keys('chat:*')

    # Get all failed queries
    all_fail_query = []

    for hash_key in all_keys:
        hash_data = r.hgetall(hash_key)

        messages = json.loads(hash_data['messages'])
        for idx in range(len(messages)):
            message = messages[idx]
            role = message['role']
            content = message['content']
            if role == 'assistant' and 'sorry' in content.lower():
                if idx - 1 >= 0:
                    prev_message = messages[idx - 1]
                    if prev_message['role'] == 'user':
                        all_fail_query.append(prev_message['content'])
    
    return all_fail_query

def get_unsolved_questions(all_fail_query):
    unsolved_questions = []

    df = pd.read_csv('./faq.csv')
    # Check if the 'question' column contains the specific question
    for query in all_fail_query:
        if not any(df['question'].str.contains(query, case=False)):
            unsolved_questions.append(query)
    
    return unsolved_questions

In [12]:
all_fail_query = get_all_failed_queries()

In [16]:
all_fail_query

['hi', 'Is there any fortune 500 alumni?']

In [24]:
get_unsolved_questions(all_fail_query)

['hi']

In [6]:
import textwrap

def perplexity_solve_question(question):
    try:
        if "rice" not in question.lower():
            question = 'At Rice university: ' + question

        perplexity = Perplexity()
        response = perplexity.search(question)

        answer = response.json_answer_text['answer']
        citations = '\n\ncitations: \n'
        for web_result_idx in range(len(response.json_answer_text['web_results'])):
            web_result = response.json_answer_text['web_results'][web_result_idx]
            url = web_result['url']
            citations += f'{[web_result_idx + 1]} {url} \n'

        answer += citations

        answer = textwrap.dedent(answer).strip().replace('\n', ' ')

        return question, answer
    except Exception as e:
        print(f"An error occurred while processing the question: {question}")
        print(f"Error message: {str(e)}")
        return None

In [7]:
question, answer = perplexity_solve_question('Is there any fortune 500 alumni?')

2023-07-29 17:37:43,818:INFO - Websocket connected


In [8]:
answer

"Yes, Rice University has alumni who have gone on to become CEOs and founders of Fortune 500 companies[3]. Some of the university's prominent alumni include individuals who have achieved success in business and have been associated with Fortune 500 companies[2][3][5]. While specific names of Fortune 500 alumni from Rice University are not mentioned in the search results, it is clear that the university has produced graduates who have gone on to achieve significant success in the business world.  citations:  [1] https://news.rice.edu/news/2022/rice-u-alums-share-fortune-500-experience-during-pioneers-innovation-chat  [2] https://www.chron.com/life/slideshow/Famous-Rice-University-alumni-202810.php  [3] https://en.wikipedia.org/wiki/Rice_University  [4] https://www.linkedin.com/posts/riceece_meet-the-rice-ece-alumni-who-built-a-fortune-activity-6982716907468582912-CNFl  [5] https://ischoolconnect.com/blog/rice-university-courses-campus-and-more/  [6] https://academic-accelerator.com/ency

In [ ]:
# Save the newly taught query into a local file to prevent teaching again nextime.
saved_query = set()